In [1]:
import pandas as pd
import numpy as np
import h5py as h5

In [4]:
x_train_store.close()
x_val_store.close()
y_train_store.close()
y_val_store.close()

In [5]:
# x_train = pd.read_hdf('train_final.h5', key="df", mode="r")
# x_val = pd.read_hdf('validation_final.h5', key="df", mode="r")
# y_train = pd.read_hdf('y_train.h5', key="df", mode="r")
# y_val = pd.read_hdf('y_validation.h5', key="df", mode="r")

# train_file = h5.File('train_final.h5', 'r')

x_train_store = pd.HDFStore('train_final.h5')
x_val_store = pd.HDFStore('validation_final.h5')
y_train_store = pd.HDFStore('y_train.h5')
y_val_store = pd.HDFStore('y_validation.h5')

INPUT_SIZE = x_train_store.get_storer('df').shape[1]

In [6]:
print("train", x_train_store.get_storer('df').shape, y_train_store.get_storer('df').shape)
print("validation", x_val_store.get_storer('df').shape, y_val_store.get_storer('df').shape)
print("columns:", INPUT_SIZE)

train [640000, 2980] [640000, 1]
validation [160000, 2980] [160000, 1]
columns: 2980


In [7]:
def generator(X_store, y_store, batch_size):
  samples_per_epoch = X_store.get_storer('df').shape[0]
  number_of_batches = samples_per_epoch/batch_size
  counter=0
  
  while 1:
    start = batch_size * counter
    stop = batch_size * (counter+1)
    X_batch = X_store.select('df', start=start, stop=stop).astype('float32')
    y_batch = y_store.select('df', start=start, stop=stop).astype('float32')
    counter += 1
    yield X_batch,y_batch

    #restart counter to yeild data in the next epoch as well
    if counter >= number_of_batches:
        counter = 0

## Training MLP

In [24]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.regularizers import l2
from keras import backend as K


In [ ]:
# from keras import backend as K
# K.tensorflow_backend._get_available_gpus()

In [ ]:
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [25]:
def rmsle(y_true, y_pred):
    return K.sqrt(K.mean(K.square(K.log(1 + y_pred) - K.log(1 + y_true)))) 

In [30]:
MODEL = "MLP-196H-64H"
HIDDEN_SIZE = (196, 64)
l2_reg = l2(0.0001)

model = Sequential()
model.add(Dense(HIDDEN_SIZE[0], input_dim=INPUT_SIZE, kernel_initializer='normal', kernel_regularizer=l2_reg, bias_regularizer=l2_reg, activation='relu'))
model.add(Dense(HIDDEN_SIZE[1], kernel_initializer='normal', kernel_regularizer=l2_reg, bias_regularizer=l2_reg, activation='relu'))
model.add(Dense(1, activation='linear', kernel_regularizer=l2_reg, bias_regularizer=l2_reg))
model.summary()

model.compile(loss=rmsle, optimizer='adam', metrics=['mse','mae'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 196)               584276    
_________________________________________________________________
dense_15 (Dense)             (None, 64)                12608     
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 65        
Total params: 596,949
Trainable params: 596,949
Non-trainable params: 0
_________________________________________________________________


In [31]:
EPOCHS = 50
BATCH_SIZE = 1000

model_checkpoint = ModelCheckpoint(f'./models/checkpoint/{MODEL}.hdf5', monitor='val_loss', verbose=1, save_best_only=True)
tensorboard_clbck = TensorBoard(log_dir='./tensorboard', histogram_freq=0, write_graph=True, write_images=True)

train_gen = generator(x_train_store, y_train_store, batch_size=BATCH_SIZE)
valid_gen = generator(x_val_store, y_val_store, batch_size=BATCH_SIZE)

# model.fit(x_train, y_train, batch_size=1000, epochs=100, validation_data=(x_val, y_val), callbacks=[model_checkpoint])

history = model.fit_generator(
    generator=train_gen,
    epochs=EPOCHS,
    verbose=1,
    steps_per_epoch=x_train_store.get_storer('df').shape[0] // BATCH_SIZE,
    validation_data=valid_gen,
    validation_steps=x_val_store.get_storer('df').shape[0] // BATCH_SIZE,
    callbacks=[model_checkpoint, tensorboard_clbck])

Epoch 1/15
640/640 [==============================] - 135s 211ms/step - loss: 0.7183 - mean_squared_error: 1401.7418 - mean_absolute_error: 14.4637 - val_loss: 0.6027 - val_mean_squared_error: 1273.7186 - val_mean_absolute_error: 13.0877

Epoch 00001: val_loss improved from inf to 0.60267, saving model to ./models/checkpoint/MLP-196H-64H.hdf5
Epoch 2/15
640/640 [==============================] - 139s 217ms/step - loss: 0.5893 - mean_squared_error: 1239.0261 - mean_absolute_error: 12.8006 - val_loss: 0.5803 - val_mean_squared_error: 1220.4673 - val_mean_absolute_error: 12.6026

Epoch 00002: val_loss improved from 0.60267 to 0.58026, saving model to ./models/checkpoint/MLP-196H-64H.hdf5
Epoch 3/15
640/640 [==============================] - 143s 224ms/step - loss: 0.5739 - mean_squared_error: 1195.7607 - mean_absolute_error: 12.4735 - val_loss: 0.5703 - val_mean_squared_error: 1190.7200 - val_mean_absolute_error: 12.3903

Epoch 00003: val_loss improved from 0.58026 to 0.57031, saving mode